# Co Simulation

In [1]:
import numpy as np
import pandas as pd

import andes
import ams

In [2]:
%matplotlib inline

In [3]:
andes.config_logger(stream_level=30)
ams.config_logger(stream_level=30)

In [4]:
curve = pd.read_csv('./../cases/Curve.csv')

In [5]:
sp1 = ams.load('./../cases/IL200_rted_db.xlsx',
               setup=True, no_output=True,
               default_config=True)

In [6]:
# set Wind and Solar to be uncontrollable
stg_wind, stg_pv = sp1.StaticGen.find_idx(keys='genfuel',
                                          values=['wind', 'solar'], allow_all=True)
sp1.StaticGen.set(src='ctrl', attr='v', idx=stg_wind, value=0.0)
sp1.StaticGen.set(src='ctrl', attr='v', idx=stg_pv, value=0.0)

# # set load levels
p0 = sp1.PQ.get(src='p0', attr='v', idx=sp1.PQ.idx.v).copy()
sp1.PQ.set(src='p0', attr='v', idx=sp1.PQ.idx.v,
           value=curve['Load'].values[0:5].mean() * p0,
        #    value=0.85 * p0,
           )

# set wind power
p0_wind = sp1.StaticGen.get(src='p0', attr='v', idx=stg_wind).copy()
sp1.StaticGen.set(src='p0', attr='v', idx=stg_wind,
                  value=curve['Wind'].values[0:5].mean() * p0_wind)

# set solar power
p0_pv = sp1.StaticGen.get(src='p0', attr='v', idx=stg_pv).copy()
sp1.StaticGen.set(src='p0', attr='v', idx=stg_pv,
                  value=curve['PV'].values[0:5].mean() * p0_pv)

stg = sp1.StaticGen.get_all_idxes()
# pg0 <- p0, relax RTED ramping constraints
sp1.StaticGen.set(src='pg0', attr='v', idx=stg,
                  value=sp1.StaticGen.get(src='p0', attr='v', idx=stg))
# sacle down StaticGen.pmin
sp1.StaticGen.set(src='pmin', attr='v', idx=stg, value=0)

sp1.RTED.run(solver='CLARABEL')

Building system matrices
Parsing OModel for <RTED>
Evaluating OModel for <RTED>
Finalizing OModel for <RTED>
<RTED> solved as optimal in 0.0443 seconds, converged in 13 iterations with CLARABEL.


True

In [7]:
sp1.RTED.dc2ac()

Parsing OModel for <ACOPF>
Evaluating OModel for <ACOPF>
Finalizing OModel for <ACOPF>
<ACOPF> solved in 0.4143 seconds, converged in 19 iterations with PYPOWER-PIPS.
<RTED> converted to AC.


True

In [8]:
s1 = sp1.to_andes(addfile='./../cases/IL200_dyn_db.xlsx',
                  setup=False,
                  no_output=True,
                  default_config=True,)

# add a Alter device to set load increase
s1.add(model='Alter',
        param_dict=dict(t=1, model='PQ', dev='PQ_1', src='Ppf',
                        attr='v', method='+', amount=0.1))
s1.add(model='Alter',
        param_dict=dict(t=1, model='PQ', dev='PQ_2', src='Ppf',
                        attr='v', method='+', amount=0.1))
s1.add(model='Alter',
        param_dict=dict(t=1, model='PQ', dev='PQ_3', src='Ppf',
                        attr='v', method='+', amount=0.1))

# bias is manually measured, in unit MW/0.1Hz
slack_bus = s1.Slack.bus.v[0]
s1.add('ACEc', param_dict=dict(bus=slack_bus,
                               bias=-45))

s1.setup()

vmax0 = s1.TGOV1NDB.get(src='VMAX', attr='v', idx=s1.TGOV1NDB.idx.v)
s1.TGOV1NDB.set(src='VMAX', attr='v', idx=s1.TGOV1NDB.idx.v,
                value=10 * vmax0)

Generating code for 1 models on 12 processes.


Following PFlow models in addfile will be overwritten: <Bus>, <PQ>, <PV>, <Slack>, <Shunt>, <Line>, <Area>
AMS system 0x3397bc200 is linked to the ANDES system 0x339a3e7e0.


True

In [9]:
sp1.dyn.send(adsys=s1, routine='RTED')

Send <vBus> to Bus.v0
Send <ug> to StaticGen.u
Send <pg> to StaticGen.p0


True

In [10]:
# Constant load
s1.PQ.config.p2p = 1
s1.PQ.config.q2q = 1
s1.PQ.config.p2z = 0
s1.PQ.config.q2z = 0

s1.PFlow.run()

True

In [11]:
s1.TDS.config.criteria = 0
s1.TDS.config.no_tqdm = 1
_ = s1.TDS.init()

/Users/jinningwang/work/andes/andes/core/model/model.py:790: ComplexWarning: Casting complex values to real discards the imaginary part
  instance.v = np.array(func(*self.s_args[name]),
/Users/jinningwang/work/andes/andes/core/model/model.py:829: ComplexWarning: Casting complex values to real discards the imaginary part
  instance.v[:] = func(*self.s_args[name])


In [12]:
s1.TDS.run()

<Alter Alter_1>: set PQ.PQ_1.Ppf.v=0.17385 at t=1. Previous value was 0.07385.
<Alter Alter_2>: set PQ.PQ_2.Ppf.v=0.11695 at t=1. Previous value was 0.01695.
<Alter Alter_3>: set PQ.PQ_3.Ppf.v=0.17947 at t=1. Previous value was 0.07947.


True

In [15]:
s1.ACEc.bias.pu_coeff

array([1.])

In [13]:
omega = s1.GENROU.omega.v[s1.GENROU.idx2uid('GENROU_47')]
bias = - 0.3 * s1.config.mva / (10 * s1.config.freq * (omega - 1))
bias

np.float64(43.715454443773346)

In [ ]:

fmin, fmax = 59.4, 60.01
tmin, tmax = 0, 8
linewidth = 1.5
genrou_idx = 'GENROU_47'

# left: short-term response, without and with thermal inertia
_ = s1.TDS.plt.plot(
    s1.GENROU.omega,
    a=s1.GENROU.idx2uid(genrou_idx),
    left=tmin, right=tmax,
    ymin=fmin, ymax=fmax,
    ytimes=s1.config.freq,
    show=False, grid=True,
    ylabel='Slack Gen. Freq. [Hz]',
)